In [43]:
# Dawar Qureshi (August, 2022)
# Script for a text based artificial intelligence chat bot for a Supermarket or a general store.
# Adapted from Amila viraj

# Chat Bot

A chatbot (Conversational AI) is a computer software that replicates human communication through text messages, audio-messages or both. It learns to do so through a variety of inputs, including Natural Language Processing (NLP).
Chatbots are especially useful for Scaling Operations, Task automation, User Engagement, Social Media Integrations,
Data generation, Connect with younger customers, to name a few. These bots are useful in industries such as healthcare, ecommerce, travel and tourism, food service and grocery stores, etc. 
The first step in creation of such a bot is defining the intents. A developer-defined intent is a task (typically a conversation). It is accessible to the user. It is used by the developer to specify probable user questions and accurate chatbot responses.

# Required Python Packages

It is recommended to use the latest version of the following packages to build the chat-box:
1) Tensorflow
2) Nltk
3) Colorama
4) Numpy
5) Scikit_learn
6) Flask

In [2]:
import json
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [27]:
# Define the intents for your specific problem

{
    "intents": [{
            "tag": "greetings",
            "patterns": ["Hi", "Hey","Heya","Hey there", "Morning","G’day!","What’s up?",  "Is anyone there?", "Hello", "Hay","How’s it going?","What’s happening?","Good afternoon","Good Morning","Good Evening","Good Day"],
            "responses": ["Hi", "Hey","Heya", "Morning","G’day!", "Hello", "Hay","Good afternoon","Good Morning","Good Evening","Good Day"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "See you soon",  "Goodbye", "Take care"],
            "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help", "Much appreciated","I really appreciate your help"],
            "responses": ["Happy to help!", "Anytime!","Glad to help","Don’t mention it", "My pleasure", "You're most welcome!"]
        },
        {
            "tag": "introducing",
            "patterns": ["Who are you?", "What are you?", "Who you are?" ],
            "responses": ["I.m Max, your bot assistant", "I'm Max, an Artificial Intelligent bot", "My name is Max"]
        },
        {
            "tag": "name",
            "patterns": ["what is your name", "what should I call you", "whats your name?"],
            "responses": ["You can call me Max.", "I'm Max!", "Just call me as Max"]
        },
        {
            "tag": "help",
            "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please", "I have a problem"],
            "responses": ["Tell me how can I assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you", "Sure, How can I help you"]
        },
        {
            "tag": "createaccount",
            "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
            "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account", "I am directing you to our website"]
        },
        {
            "tag": "complaint",
            "patterns": ["I have a complaint", "I want to raise a complaint", "there is a complaint about a service", "I am not happy about the service", "I have a problem with the service"],
            "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused", "Sorry for the issues, please mention your complaint"]
        }
    ]
}

{'intents': [{'tag': 'greetings',
   'patterns': ['Hi',
    'Hey',
    'Heya',
    'Hey there',
    'Morning',
    'G’day!',
    'What’s up?',
    'Is anyone there?',
    'Hello',
    'Hay',
    'How’s it going?',
    'What’s happening?',
    'Good afternoon',
    'Good Morning',
    'Good Evening',
    'Good Day'],
   'responses': ['Hi',
    'Hey',
    'Heya',
    'Morning',
    'G’day!',
    'Hello',
    'Hay',
    'Good afternoon',
    'Good Morning',
    'Good Evening',
    'Good Day']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'See you soon',
    'Goodbye',
    'Take care'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help',
    'Much appreciated',
    'I really appreciate your help'],
   'responses': ['Happy to help!',
    'Anytime!',
    'Glad to help',
    'Don’t mention it',
    'My pleasure',
    "You're most w

# Load the data

Creat and load the JSON file

Training_sentences variable includes all of the training data i.e. the sample messages in each intent category and the training_labels variable contains all of the target labels that correspond to each training data. Scikit's LabelEncoder( ) converts the target labels into model understandable form. 

In [30]:
with open('intents.json', 'w') as f:
    json.dump(intents, f)

In [32]:
with open("intents.json") as file: 
    data = json.load(file)
    
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [33]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

Next vectorize the text data set using  "Tokenizer" class, which enables  to restrict the amount of vocabulary to a certain number. If this class is used for text pre-processing, all punctuation is removed by default, converting the texts into space-separated word sequences that are then divided into lists of tokens. After that, they will be indexed or vectorized. To deal with out of vocabulary words (tokens) during inference, we may additionally include "oov token," which is a value for "out of token."

In [34]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# Model Training

For defining the Neural Network architecture, “Sequential” model class of Keras is used. To train the model, simply call the fit method. Save the trained model, fitted tokenizer object, and fitted label encoder object for using during the inference process.

In [37]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [38]:
epochs = 700
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/700
2/2 [==============================] - 0s 34ms/step - loss: 2.0820 - accuracy: 0.0588
Epoch 2/700
2/2 [==============================] - 0s 13ms/step - loss: 2.0786 - accuracy: 0.0588
Epoch 3/700
2/2 [==============================] - 0s 14ms/step - loss: 2.0760 - accuracy: 0.1765
Epoch 4/700


2022-09-19 19:02:55.419346: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 10ms/step - loss: 2.0737 - accuracy: 0.3922
Epoch 5/700
2/2 [==============================] - 0s 9ms/step - loss: 2.0713 - accuracy: 0.3529
Epoch 6/700
2/2 [==============================] - 0s 15ms/step - loss: 2.0695 - accuracy: 0.3137
Epoch 7/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0675 - accuracy: 0.3137
Epoch 8/700
2/2 [==============================] - 0s 9ms/step - loss: 2.0655 - accuracy: 0.3137
Epoch 9/700
2/2 [==============================] - 0s 14ms/step - loss: 2.0636 - accuracy: 0.3137
Epoch 10/700
2/2 [==============================] - 0s 15ms/step - loss: 2.0619 - accuracy: 0.3137
Epoch 11/700
2/2 [==============================] - 0s 7ms/step - loss: 2.0599 - accuracy: 0.3137
Epoch 12/700
2/2 [==============================] - 0s 8ms/step - loss: 2.0578 - accuracy: 0.3137
Epoch 13/700
2/2 [==============================] - 0s 10ms/step - loss: 2.0555 - accuracy: 0.3137
Epoch 14/700
2/2 [===============

In [40]:
# save the trained model
model.save("chat_bot")

import pickle

# save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_bot/assets


# Implementation 

In [41]:

import numpy as np
import colorama 
colorama.init()
from colorama import Fore, Style, Back
import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load the trained model
    model = keras.models.load_model('chat_bot')

    # load the tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load the label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hello
1/1 [==============================] - 0s 77ms/step
ChatBot: Heya
User: 

2022-09-19 19:04:04.899712: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


What's your name 
1/1 [==============================] - 0s 18ms/step
ChatBot: Just call me as Max
User: I have a complaint 
1/1 [==============================] - 0s 17ms/step
ChatBot: Sorry for the issues, please mention your complaint
User: I need to create an account
1/1 [==============================] - 0s 19ms/step
ChatBot: You can just easily create a new account from our web site
User: thank you
1/1 [==============================] - 0s 17ms/step
ChatBot: Glad to help
User: bye
1/1 [==============================] - 0s 18ms/step
ChatBot: See you later
User: quit
